# Testing environment to transfer functions to scipt

In [ ]:
from collections import defaultdict
import os
os.chdir("/Users/oliver/Documents/GitHub/generic-buy-now-pay-later-project-group-19/scripts")
import sys
import argparse
import re
# ... TODO: Add to this as necessary

# External Libraries
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, DataFrame
# ... TODO: Add to this as necessary

# Our Modules
from utilities.log_utilities import logger
import utilities.print_utilities as PRINT
import utilities.read_utilities as READ
import utilities.clean_utilities as CLEAN
import utilities.agg_utilities as AGG
import utilities.write_utilities as WRITE

In [ ]:
spark = (
        SparkSession.builder.appName("MAST30034 Project 2")
        .config("spark.sql.repl.eagerEval.enabled", True) 
        .config("spark.sql.parquet.cacheMetadata", "true")
        .config("spark.sql.session.timeZone", "Etc/UTC")
        .config("spark.driver.memory", "4g")
        .getOrCreate()
    )

In [ ]:
os.getcwd()

In [ ]:
data_dict = READ.read_data(spark, "../data/tables")

In [ ]:
CLEAN.clean_data(spark, data_dict)

In [ ]:
data_dict.keys()

In [ ]:
data_dict_2 = data_dict.copy()

In [ ]:
spark.catalog.clearCache()

In [ ]:
AGG.compute_aggregates(spark, data_dict)

In [ ]:
data_dict.keys()

In [ ]:
from pyspark.sql import functions as F
import pandas as pd
from datetime import datetime

def compute_merchant_metric(spark: SparkSession, merchant_sales: DataFrame,
                           merchant: DataFrame) -> DataFrame:
    
    
    # This part is taking a while 
    date_range = merchant_sales.select(F.min(F.col("order_datetime")), 
                                       F.max(F.col("order_datetime"))
                                      ).first()
    
    min_date, max_date = (datetime.strptime(date_range[0], "%Y-%m-%d"), 
                          datetime.strptime(date_range[1], "%Y-%m-%d"))
    
    num_days = (max_date - min_date).days
    
    # Group first to reduce the table size before joining
    merchant_daily_sales = merchant_sales.groupby('merchant_abn').agg(
        (F.sum(F.col('sales_revenue')) / num_days).alias('avg_daily_rev'),
        (F.sum(F.col('sales_revenue')) / F.sum(F.col('no_orders'))).alias('avg_value_per_order'),
        (F.sum(F.col('no_orders')) / num_days).alias('avg_daily_order')
    )
    
    merchant_daily_sales = merchant.join(
        merchant_daily_sales, 
        on=["merchant_abn"],
        how='left'
    ).toPandas()
    
    
    merchant_daily_sales['avg_daily_commission'] = merchant_daily_sales['avg_daily_rev'] * (merchant_daily_sales['take_rate']/100)
    merchant_daily_sales['avg_commission_per_order'] = merchant_daily_sales['avg_value_per_order'] * (merchant_daily_sales['take_rate']/100)
    
    return merchant_daily_sales
    

In [ ]:
data_dict['merchant_sales'].columns

In [ ]:
data_dict['merchant_summary'] = compute_merchant_metric(spark, data_dict['merchant_sales'], data_dict['merchants'])

In [ ]:
date_range = data_dict['merchant_sales'].select(F.min(F.col("order_datetime")), F.max(F.col("order_datetime"))).first()